In [ ]:
import sys
import importlib
import datetime

import numpy as np
import pandas as pd
import xarray as xr
import pvlib as pv
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from distributed import LocalCluster, Client

In [ ]:
%matplotlib inline

In [ ]:
sys.path.append('/home/travis/python_code/letkf_forecasting/')
import letkf_forecasting as lf

In [ ]:
import prepare_sat_data as prep

In [ ]:
sensor_data = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_data.h5')
sensor_CI = pd.read_hdf('/home/travis/python_code/letkf_forecasting/sensor_CI.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_loc.h5')

In [ ]:
sensor_loc.sort_values(by='id', inplace=True)

In [ ]:
csi_15 = pd.read_hdf('./data/for_15/csi_15.h5')
U_15 = pd.read_hdf('./data/for_15/U_fine.h5')
V_15 = pd.read_hdf('./data/for_15/V_fine.h5')
U_shape = np.load('./data/for_15/U_fine_shape.npy')
V_shape = np.load('./data/for_15/V_fine_shape.npy')

In [ ]:
## filter winds
filter_len = 20*4
U_smooth = U_15.values.reshape(
    [U_15.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U_15.index)

V_smooth = V_15.values.reshape(
    [V_15.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V_15.index)

In [ ]:
x_sat = np.load('./data/for_15/x_fine.npy')
y_sat = np.load('./data/for_15/y_fine.npy')
domain_shape = np.load('./data/for_15/fine_shape.npy')

In [ ]:
x_range = x_sat.reshape(domain_shape)[0, :]
y_range = y_sat.reshape(domain_shape)[:, 0]

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
U_max = U_15.values.max() # know U is positive
V_max = abs(V_15.values.min()) # know V is negative

left = int(U_max*60*30/250) + 20
right = 20

up = int(V_max*60*30/250) + 20
down = 20

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
n_workers = 20
cluster = LocalCluster(n_workers=n_workers, scheduler_port=7001, diagnostics_port=7002)
client = Client(cluster)

In [ ]:
csi_crop = csi_15[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
# Use actual satellite image as Truth to find error statistics
# forecast and analysis trace p
# Also do the same by converting cloudiness to csi and compare to sensor
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 1 #was 5
localization_letkf = 1
sat_inflation = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 #0.05 #0.001 <- good
ens_size = 40
wind_sigma = (.5, .5)
wind_size = 2

start_time = '2014-04-15 12:30:00' #11:00:00 is not a bad start
end_time = '2014-04-15 13:30:00' #Gets boring shortly after 14:00:00

analysis, background, advected, sat_time_range, trace_back, trace_analy = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_crop, U_shape=U_crop_shape, V=V_crop, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,                                                                                                                                                             
    client=client)

In [ ]:
analysis_only_sat = analysis.copy()
background_only_sat = background.copy()
advected_only_sat = advected.copy()
sat_time_range_only_sat = sat_time_range.copy()

In [ ]:
plt.figure()
plt.plot(trace_back[1:])
plt.plot(trace_analy[1:])
plt.legend(['background', 'analysis'])

In [ ]:
# 0.05
shape = crop_shape
for t in range(sat_time_range.size):
    this_time = sat_time_range[t]
    
    vmin=0
    vmax=1.2
    
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        background[t, 2:].reshape(shape) ,
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Background: ' + str(this_time))
    plt.axis('equal')
    
#     plt.figure()
#     im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
#         advected[t].reshape(shape),
#         cmap='Blues', vmin=vmin, vmax=vmax)
#     plt.colorbar(im)
#     plt.title('advected: ' + str(this_time))
#     plt.axis('equal')
    
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        (analysis[t, 2:]).reshape(shape),
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Analysis: ' + str(this_time))
    plt.axis('equal')

In [ ]:
#0.001
shape = crop_shape
for t in range(sat_time_range.size):
    this_time = sat_time_range[t]
    
    vmin=0
    vmax=1.2
    
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        background[t, 2:].reshape(shape) ,
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Background: ' + str(this_time))
    plt.axis('equal')
    
#     plt.figure()
#     im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
#         advected[t].reshape(shape),
#         cmap='Blues', vmin=vmin, vmax=vmax)
#     plt.colorbar(im)
#     plt.title('advected: ' + str(this_time))
#     plt.axis('equal')
    
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        (analysis[t, 2:]).reshape(shape),
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Analysis: ' + str(this_time))
    plt.axis('equal')

In [ ]:
cloudiness_14 = pd.read_hdf('./data/for_15/cloudiness_14.h5')
cloudiness_15 = pd.read_hdf('./data/for_15/cloudiness_15.h5')

In [ ]:
corrected_cloudiness = (cloudiness_15 - cloudiness_14).clip(lower=0)
cc_crop = corrected_cloudiness[flat_crop_indicies]
cc_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
analysis[0]

In [ ]:
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 5
localization = 2
sat_inflation = 1
sensor_inflation = 1.2

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.01 #0.01 #0.05
sensor_sig = 0.05 #0.1 #0.05
ens_size = 40
wind_sigma = (.5, .5)
wind_size = 2
CI_sigma = .0001
coeff_sigma = np.array([.3, .5])
SUNY_coeff = np.array(np.array([2.36, -6.2, 6.22, -2.63, -0.58, 1]))

tus = pv.location.Location(32.2, -111, 'US/Arizona', 700,'Tucson')

cloud_height = 8222

start_time = '2014-04-15 12:45:00' #11:00:00 is not a bad start
end_time = '2014-04-15 13:45:00' #Gets boring shortly after 14:00:00

analysis, analysis_error, background, background_error, advected, time_range, trace_back, trace_analy = lf.main(
    sat=cc_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_crop, U_shape=U_crop_shape, V=V_crop, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,                                                                                                                                                             
    client=client,
    sensor_data=sensor_CI,
    sensor_loc=sensor_loc,
    sensor_sig=sensor_sig,
    sensor_inflation=sensor_inflation,
    CI_sigma=CI_sigma, coeff_sigma=coeff_sigma, location=tus,
    cloud_height=cloud_height, sat_azimuth=goes15_azimuth,
    sat_elevation=goes15_elevation)

In [ ]:
shape = crop_shape
for t in range(time_range.size):
    this_time = time_range[t]
    
    vmin=0
    vmax=1.05
    
#     vmin=None
#     vmax=None

    this_coeff_pert = np.array(
        [0,0,0,0, background[t, 1] - background[t, 0], background[t, 0]])
    this_background = np.polyval(SUNY_coeff + this_coeff_pert, background[t, 2 + 2:].reshape(shape))
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        this_background,
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Background: ' + str(this_time))
    plt.axis('equal')
    
#     plt.figure()
#     im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
#         advected[t].reshape(shape),
#         cmap='Blues', vmin=vmin, vmax=vmax)
#     plt.colorbar(im)
#     plt.title('advected: ' + str(this_time))
#     plt.axis('equal')
    
    this_coeff_pert = np.array(
        [0,0,0,0, analysis[t, 1] - analysis[t, 0], analysis[t, 0]])
    this_analysis = np.polyval(SUNY_coeff + this_coeff_pert, analysis[t, 2 + 2:].reshape(shape))
    plt.figure()
    im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
        this_analysis,
        cmap='Blues', vmin=vmin, vmax=vmax)
    plt.colorbar(im)
    plt.title('Analysis: ' + str(this_time))
    plt.axis('equal')
    
#     plt.figure()
#     im = plt.pcolormesh(x_crop.reshape(shape), y_crop.reshape(shape),
#         this_analysis-this_background,
#         cmap='Blues', vmin=vmin, vmax=vmax)
#     plt.colorbar(im)
#     plt.title('Analysis: ' + str(this_time))
#     plt.axis('equal')

In [ ]:
plt.figure()
plt.plot(np.sqrt(trace_back[1:]))
plt.plot(np.sqrt(trace_analy[1:]))
plt.legend(['background', 'analysis'])

In [ ]:
analysis_rmse = np.sqrt((analysis_error**2).mean(axis=1))
background_rmse = np.sqrt((background_error**2).mean(axis=1))
plt.figure()
plt.plot(background_rmse[1:])
plt.plot(analysis_rmse[1:])
plt.legend(['background', 'analysis'])

In [ ]:
plt.figure()
plt.plot(analysis_error)

In [ ]:
plt.figure()
sensor_CI[sensor_loc[sensor_loc.test==True].id].ix['2014-04-15 12:30' : '2014-04-15 13:30'].plot()

In [ ]:
plt.figure()
plt.plot(np.sqrt((background_error**2).mean(axis=1))[1:])
plt.plot(np.sqrt((analysis_error**2).mean(axis=1))[1:])
plt.legend(['background', 'analysis'])

In [ ]:
U_shape

In [ ]:
U_15.shape

In [ ]:
plt.figure()
im = plt.pcolormesh(
    U_15.iloc[3].reshape(U_shape),
    cmap='Blues')
plt.colorbar(im)
plt.axis('equal')

In [ ]:
V_crop.values.max()

In [ ]:
U_crop.values.max()

In [ ]:
from functools import partial

In [ ]:
start_time = '2014-04-15 12:30:00' #11:00:00 is not a bad start
end_time = '2014-04-15 13:30:00' #Gets boring shortly after 14:00:00

assimilation_grid_size = 5


short_fun_d = partial(lf.main,
    sat=sat_15.isel(west_east=x_crop, south_north=y_crop),
    wind=wind_15_smooth.isel(west_east=x_crop, west_east_stag=x_crop_stag,
                             south_north=y_crop, south_north_stag=y_crop_stag),
    sensor_data=sensor_CI,
    sensor_loc=sensor_loc,
    start_time=start_time, end_time=end_time, dx=dx, dy=dy,
    C_max=C_max,
    assimilation_grid_size=assimilation_grid_size,
    localization_length=localization, inflation=inflation,
    sat_sig=sat_sig, sensor_sig=sensor_sig, ens_size=ens_size,
    wind_sigma=wind_sigma, wind_size=wind_size, CI_sigma=CI_sigma,
    location=tus, cloud_height=cloud_height, sat_azimuth=goes15_azimuth,
    sat_elevation=goes15_elevation, client=client)

In [ ]:
%lprun -f lf.main short_fun_d()

In [ ]:
np.arange(16).reshape(4,4).mean()

In [ ]:
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7

start_time = '2014-04-15 12:30:00' #11:00:00 is not a bad start
end_time = '2014-04-15 13:30:00' #Gets boring shortly after 14:00:00

u_pert = 1
v_pert = 0

x_crop_stag = slice(x_crop.start - 1, x_crop.stop)
y_crop_stag = slice(y_crop.start - 1, y_crop.stop)

for u_pert in np.linspace(-5, 5, 10):
    for v_pert in np.linspace(-2, 2, 5):
        print('u_pert: ' + str(u_pert))
        print('v_pert: ' + str(v_pert))
        lf.just_advection(
            sat=sat_15.isel(west_east=x_crop, south_north=y_crop),
            wind=wind_15_smooth.isel(west_east=x_crop, west_east_stag=x_crop_stag,
                                     south_north=y_crop, south_north_stag=y_crop_stag),
            u_pert=u_pert, v_pert=v_pert,
            start_time=start_time, end_time=end_time, dx=dx, dy=dy,
            C_max=C_max)

In [ ]:
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7

start_time = '2014-04-15 12:30:00' #11:00:00 is not a bad start
end_time = '2014-04-15 13:30:00' #Gets boring shortly after 14:00:00

u_pert = 1
v_pert = 0

x_crop_stag = slice(x_crop.start - 1, x_crop.stop)
y_crop_stag = slice(y_crop.start - 1, y_crop.stop)

In [ ]:
for i in sensor_loc[sensor_loc.test==False].id:
    plt.figure()
    plt.plot(sensor_data[sensor_data.id==i]['clearsky_profile'].ix['2014-04-15'])
    plt.plot(sensor_data[sensor_data.id==i]['ghi'].ix['2014-04-15'])